### Kütüphaneler

In [33]:
# Gerekli kütüphaneleri içe aktarın
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense

### Veri Seti

In [34]:
data = pd.read_csv("data/final_data.csv")

In [35]:
data.sample(9)

,Unnamed: 0,Longitude,Latitude,Accident_Severity,Number_of_Vehicles,Number_of_Casualties,1st_Road_Class,Road_Type,Speed_limit,Junction_Control,Weather_Conditions,Special_Conditions_at_Site,Urban_or_Rural_Area,Year,Darkness_Presence,Road_Surface_Conditions_Ordinal,datetime
899052,899055,-0.021451,51.409915,3,1,1,6,Single carriageway,30,Giveway or uncontrolled,0,No Special Conditions or Not Specified,1,2011,0,5,2011-07-01T17:08:00.000000Z
293761,293761,-1.136518,52.952418,2,3,6,3,Single carriageway,30,Not Specified,0,No Special Conditions or Not Specified,1,2006,1,5,2006-03-03T19:58:00.000000Z
1212361,1212369,-2.222810,53.692066,3,1,1,6,Single carriageway,30,Giveway or uncontrolled,0,No Special Conditions or Not Specified,1,2013,0,5,2013-07-18T13:13:00.000000Z
614872,614874,-1.617277,54.950988,3,2,1,4,Single carriageway,30,Automatic traffic signal,0,No Special Conditions or Not Specified,1,2009,0,5,2009-05-19T13:00:00.000000Z
983494,983500,0.231093,51.487216,3,2,1,6,Single carriageway,30,Not Specified,0,No Special Conditions or Not Specified,1,2011,0,5,2011-03-14T08:10:00.000000Z
1234846,1234854,-1.147149,53.425946,3,1,2,3,Single carriageway,60,Not Specified,0,No Special Conditions or Not Specified,2,2013,2,4,2013-11-17T00:40:00.000000Z
869262,869265,-1.831646,50.728601,3,3,1,3,Single carriageway,30,Giveway or uncontrolled,0,No Special Conditions or Not Specified,1,2010,0,5,2010-06-24T13:35:00.000000Z
285017,285017,-2.052942,52.329756,3,4,1,3,Dual carriageway,60,Automatic traffic signal,0,No Special Conditions or Not Specified,1,2006,0,4,2006-08-19T19:25:00.000000Z
1387882,1387892,-1.879747,52.505647,3,1,4,2,Dual carriageway,50,Not Specified,0,No Special Conditions or Not Specified,1,2014,1,5,2014-09-26T03:00:00.000000Z


### Tarih verisini yıl, ay, gün olarak ayırma

In [36]:
data["datetime"] = pd.to_datetime(data["datetime"])

In [37]:
data["Year"] = data["datetime"].dt.year
data["Month"] = data["datetime"].dt.month
data["Day"] = data["datetime"].dt.day

### Yapay Sinir Ağı Modeli

In [53]:
# Veri setinin bölünmesi
X = data[["Year", "Month", "Day", "Longitude", "Latitude", "Number_of_Casualties", "Speed_limit", "Weather_Conditions", "Urban_or_Rural_Area", "Darkness_Presence", "Road_Surface_Conditions_Ordinal"]]
y = data[["Longitude", "Latitude"]]

# Eğitim - Test kümeleri
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Ölçeklendirme
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Yapay sinir ağı modeli
model = Sequential()
model.add(Dense(8, input_dim=X_train.shape[1], activation="relu"))
model.add(Dense(16, activation="relu"))
model.add(Dense(2)) 

# Modelin derlenmesi
model.compile(loss="mean_squared_error", optimizer="adam")

# Modelin eğitimi
model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test))


Epoch 1/5
36740/36740 [==============================] - 81s 2ms/step - loss: 11.7898 - val_loss: 6.4670e-04
Epoch 2/5
36740/36740 [==============================] - 80s 2ms/step - loss: 0.0010 - val_loss: 9.4094e-05
Epoch 3/5
36740/36740 [==============================] - 69s 2ms/step - loss: 5.4181e-04 - val_loss: 0.0054
Epoch 4/5
36740/36740 [==============================] - 69s 2ms/step - loss: 4.9239e-04 - val_loss: 7.4106e-06
Epoch 5/5
36740/36740 [==============================] - 70s 2ms/step - loss: 4.0071e-04 - val_loss: 2.6365e-05


### Tahmin

In [54]:
predictions = model.predict(X_test)

9185/9185 [==============================] - 13s 1ms/step


#### Test

In [55]:
# Örnek için rastgele bir tahmin alınması
sample_index = 158616
predicted_location = predictions[sample_index]
real_location = y_test.iloc[sample_index]

print("Gerçek Konum:", real_location)
print("Tahmin Edilen Konum:", predicted_location)


Gerçek Konum: Longitude     0.268551
Latitude     51.135392
Name: 1002983, dtype: float64
Tahmin Edilen Konum: [ 0.2683884 51.127556 ]


### MSE

In [56]:
from sklearn.metrics import mean_squared_error

mse = mean_squared_error(y_test, predictions)
print("Ortalama Kare Hata (MSE):", mse)


Ortalama Kare Hata (MSE): 2.636525058482364e-05


In [65]:
# user_input_year = 2010
# user_input_month = 5
# user_input_day = 5
# user_input_number_of_casualties = 1
# user_input_speed_limit = 30
# user_input_weather_condition = 0
# user_input_urban_or_rural = 1
# user_input_darkness_presence = 0
# user_input_road_surface_conditions = 5


# user_input_features = np.array([[user_input_year, user_input_month, user_input_day, 
#                                  user_input_number_of_casualties,
#                                  user_input_speed_limit,
#                                  user_input_weather_condition,
#                                  user_input_urban_or_rural,
#                                  user_input_darkness_presence,
#                                  user_input_road_surface_conditions, 
#                                  0.268551, 
#                                  51.135392]])  

# user_input_features = scaler.transform(user_input_features)  

# predicted_location = model.predict(user_input_features)

# print("Tahmin Edilen Konum:", predicted_location)


1/1 [==============================] - 0s 18ms/step
Tahmin Edilen Konum: [[-1.6314106 69.28973  ]]


c:\Users\kdrer\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
